
# Getting Started with AMC Insights
A starting point for new users to the service. Contains a walkthrough of onboarding your first client as well as creating and invoking AMC workflows.
#### Import Libraries

In [ ]:
import json
from library.workflow_interface import WorkflowInterface
from library.customer_interface import CustomerInterface

#### Define Global Configurations
These configuration parameters are needed in order to interact with the correct architecture resources. They serve as constructors for both interface modules.

In [ ]:
# Change to your Default Environment
ENV = "dev"

# This is the same value passed in the 'data_pipeline_parameters' team of the ddk.json file
TEAM_NAME = "<INSERT TEAM NAME>"

# Setting these interface classes allow calling the various WFM and TPS methods
workflow_interface = WorkflowInterface(TEAM_NAME=TEAM_NAME, ENV=ENV)
customer_interface = CustomerInterface(TEAM_NAME=TEAM_NAME, ENV=ENV)

#### Step 1: Onboard A New Customer

The Tenant Provisioning Service (TPS) is used for onboarding new customers to each team space. Each customer is tied 1 to 1 with an AMC instance.

The following notebook cells will define your customer AMC instance details and onboard your customer.

##### Define Customer Details
Edit and run the cell below to define your customer details.

In [ ]:
# This is the same value passed in the 'data_pipeline_parameters' dataset of the ddk.json file
dataset_name = "<INSERT DATASET NAME>" 

# The AMC Instance information can be found on your AMC UI page
amc_api_endpoint = "<ENTER AMC API ENDPOINT URL>"
amc_s3_bucket_name = "<ENTER AMC S3 BUCKET NAME>"
amc_data_upload_acct = "<ENTER AMC DATA UPLOAD ACCOUNT ID>"

## The AMC Instance AWS region (found in the API Endpoint URL) e.g. 'us-east-1'
# This should be the same as your deployment region
amc_instance_region = "<ENTER AMC INSTANCE REGION>"

# OPTIONAL: Change to your desired Customer ID (keep less than 25 characters) 
customerId = "testdemocustomer"

# OPTIONAL: Change to your desired Customer Prefix
customerPrefix = "testdemocustomer"

# OPTIONAL: Change to your desired Customer Name
customerName = "DemoCustomer"

# OPTIONAL: Can Be <ENDEMIC or NON-ENDEMIC>
endemicType = "ENDEMIC"

##### Confirm Customer Details

Run the cell below to format the defined customer settings.

In [ ]:
customerDetails = {
    "customerPrefix": customerPrefix,
    "customerName": customerName, 
    "endemicType": endemicType, 
    "region": amc_instance_region,
    "amc":{
        "amcDatasetName": dataset_name,
        "amcApiEndpoint": amc_api_endpoint,
        "amcOrangeAwsAccount": amc_data_upload_acct,
        "amcS3BucketName": amc_s3_bucket_name
    }
}

print(f'customerId = {customerId}')
print(f'customerDetails = {json.dumps(customerDetails, indent=4)}')

##### Submit Customer Onboarding

Run the cell below to start the process of onboarding your customer and creating the AMC S3 Bucket if it does not already exist.

In [ ]:
response = customer_interface.onboard_customer(
    customerId=customerId, 
    customerDetails=customerDetails
    )

print(response)

_Wait a few minutes for the cloudformation stack to be deployed BEFORE moving to Step #2._

_You Can Verify the Status by going to AWS Step Functions and waiting until the state machine named tps-&lt;TeamName&gt;-initialize-amc has 1 Succeeded Execution Status._

#### Step 2: Create a Workflow Record 

The Workflow Manager Service (WFM) is used to manage and schedule AMC workflows. The following notebook cells will walk through the process of creating a new workflow record.

The query used here is the Time To Conversion Query from the Interactive Query Library (IQL) in the AMC UI.

The query finds out how long it takes for your customers to convert after last seeing your ad. You can use this information to adjust the duration of campaign and promotion to maximize sales. In our amazon_attributed_events_by_conversion_time and amazon_attributed_events_by_traffic_time tables, we report up to 14 days after the customers’ last exposure to your ad.

##### Define Workflow Query

Run the cell below to define the query that will be used in the workflow.


In [ ]:
amc_query = """
SELECT
      advertiser,
      campaign,
        ( 
            CASE WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 60 THEN
                '1 | < 1 MIN'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 600 THEN
                '2 | 1 - 10 MIN'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 1800 THEN
                '3 | 10 - 30 MIN'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 3600 THEN
                '4 | 30 - 60 MIN'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 7200 THEN
                '5 | 1 - 2 HRS'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 43200 THEN
                '6 | 2 - 12 HRS'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 86400 THEN
                '7 | 12 - 24 HRS'
                WHEN SECONDS_BETWEEN (impression_dt,
                    conversion_event_dt) <= 604800 THEN
                '8 | 1 - 7 DAYS'
            ELSE
                '9 | 7+ DAYS'
END
) AS time_to_conversion,
        SUM(purchases) AS purchases,
        SUM(total_purchases) AS total_brand_purchases
FROM
    amazon_attributed_events_by_conversion_time
    
GROUP BY 1,2,3
"""

print (amc_query)

__NOTE__: This is just one example of a workflow query you can run on your AMC Instance. Refer to the Interactive Query Library (IQL) in the AMC UI for a list of other queries for different use cases. Queries can also be customized for you unique use case as well.

##### Define Workflow Details

Run the cell below to define the workflow settings.

In [ ]:
#OPTIONAL: Change to your desired Workflow ID
workflowId = "time_to_conversion_v1"

# OPTIONAL: Change to your desired Workflow Name
workflowName = "time_to_conversion"

# OPTIONAL: Change to your desired Workflow Description
workflowDescription = "Runs the time_to_conversion workflow."

### Default Payload
## The following parameters are saved to the Workflow and will be used in the absense of invocation overrides
# OPTIONAL: Change to your desired Time Window End
timeWindowEnd = "today(-1)"

# OPTIONAL: Change to your desired Time Window Start
timeWindowStart = "today(-91)"

# OPTIONAL: Change to your desired Time Window Type
timeWindowType = "EXPLICIT"

# OPTIONAL: Change to your desired Workflow Executed Date
workflowExecutedDate = "now()"


##### Confirm Workflow Details

Run the cell below to format the defined workflow settings.

In [ ]:
workflowId = workflowId
workflowDetails = {
    "metadata": {
        "workflowName": workflowName,
        "description": workflowDescription,
    },
    "sqlQuery": amc_query,
    "defaultPayload": {
            "timeWindowEnd": timeWindowEnd,
            "timeWindowStart": timeWindowStart,
            "timeWindowType": timeWindowType,
            "workflowExecutedDate": workflowExecutedDate
    }
}

print(f'workflowId = {workflowId}')
print(f'workflowDetails = {json.dumps(workflowDetails, indent=4)}')

##### Submit Workflow Create

Run the cell below to create and save the workflow details.

In [ ]:
response = workflow_interface.set_workflow_record(
    workflowId=workflowId,
    customerId=customerId,
    workflowDetails=workflowDetails
    )

print(response)

#### Step 3: Invoke the Workflow to be Executed Ad Hoc

Saved workflows can be executed adhoc or on a schedule. In the following steps we will invoke the workflow we just created once adhoc.

Refer to the Workflow_Schedules notebook for information on how to schedule saved workflows.

##### Submit Workflow Invoke Request

Run the cell below to submit the workflow to be processed adhoc.

In [ ]:
response = workflow_interface.invoke_workflow(
    workflowId=workflowId, 
    customerId=customerId
    )

print(response)

#### Your workflow is now being processed and executed
Once the workflow has run, data will be uploaded to your AMC S3 Bucket with the results of the workflow query and then processed through the data lake. 

#### **Next Steps**
After going through the guide above, the following notebooks provide templates for interacting with the microservice solution.

**Customers**: managing and onboarding new customers to the service.
**Workflows**: managing and invoking workflows for single customers.  
**Workflow_Schedules**: managing workflow schedules for single customers.  
**Workflow_Library**: managing workflow and schedules for multiple customers.  

The reference folder located in this deployed instance also contains additional documentation